In [1]:
import pandas as pd
import re
import numpy as np

# Read in the data
# job_requirements_scores = pd.read_csv('job_requirement_matching_scores_all.csv')
# job_seekers_scores = pd.read_csv('job_seeker_matching_scores_all.csv')
match_company_df=pd.read_csv('data/result3-1.csv',dtype=str)
match_seekers_df=pd.read_csv('data/result3-2.csv',dtype=str)
#job_requirements_df = pd.read_csv('根据id获取详细数据.csv')
#job_seekers_df = pd.read_csv('根据人才id获取详细数据.csv')
job_requirement = pd.read_csv('data/result1-1.csv')
#ob_requirements_df_sort = job_requirements_df.sort_values(by=['招聘信息id'],ascending=True)
#job_seekers_df_sort = job_seekers_df.sort_values(by=['求职者id'],ascending=True)

In [2]:
# 显示match_seekers_df中有多少个不同的求职者id
print(len(match_seekers_df['求职者id'].unique()))

120


def load_seeker_data(filename):
    """
    读取满意度csv文件，返回一个字典，键为求职者id，值为按满意度降序排列的公司id列表和所需要的人数
    """
    data = {}
    for index, row in filename.iterrows():
        user_id = int(row[0].rstrip(','))
        company_ids = [int(x.rstrip(',')) for x in row[1:]]
        data[user_id] = {'company_ids' : company_ids}
    return data
def load_company_data(filename):
    """
    读取满意度csv文件，返回一个字典，键为公司id，值为按满意度降序排列的求职者id列表和所需要的人数
    """
    data = {}
    for index, row in filename.iterrows():
        company_id = int(row[0].rstrip(','))
        user_ids = [int(x.rstrip(',')) for x in row[1:-1]]
        if user_ids == []:
            user_ids = [0]
        num_positions = int(row[-1].rstrip(','))
        data[company_id] = {'user_ids': user_ids, 'num_positions': num_positions}
    return data


In [3]:
# 从job_requirement 中获取与招聘信息id对应的岗位数量，并存到match_company_df中
job_requirement = job_requirement[['招聘信息id','岗位数量']]
job_requirement['招聘信息id'] = job_requirement['招聘信息id'].astype(str)
match_company_df = pd.merge(match_company_df,job_requirement,on='招聘信息id',how='left')
# 将岗位数量中的空值替换为0,int类型
match_company_df['岗位数量'] = match_company_df['岗位数量'].fillna(0)
match_company_df['岗位数量'] = match_company_df['岗位数量'].astype(int)
# 保存到文件
match_company_df.to_csv('data/result3-1+.csv',index=False,encoding='utf-8-sig')

In [4]:
import csv

def load_seeker_data(filename):
    """
    读取满意度csv文件，返回一个字典，键为求职者id，值为按满意度降序排列的公司id列表和所需要的人数
    """
    data = {}
    with open(filename, 'r') as f:
        # 使用列表切片去掉第一行
        reader = csv.reader(f)
        next(reader) 
        for line in reader:
            user_id = int(line[0])
            company_id = int(line[1])
            company_name = line[2]
            satisfaction = float(line[3])
            if user_id not in data:
                data[user_id] = {'company_ids': [], 'company_name':[], 'satisfaction': []}
            data[user_id]['company_ids'].append(company_id)
            data[user_id]['company_name'].append(company_name)
            data[user_id]['satisfaction'].append(satisfaction)
    return data


def load_company_data(filename):
    """
    读取满意度csv文件，返回一个字典，键为公司id，值为按满意度降序排列的求职者id列表和所需要的人数
    """
    data = {}
    with open(filename, 'r') as f:
        # 使用列表切片去掉第一行
        reader = csv.reader(f)
        next(reader) 
        for line in reader:
            user_id = int(line[1])
            company_id = int(line[0])
            match = float(line[2])
            num_position = int(line[3])
            if company_id not in data:
                data[company_id] = {'user_id': [], 'match': [], 'num_position': num_position}
            data[company_id]['user_id'].append(user_id)
            data[company_id]['match'].append(match)
            data[company_id]['num_position'] = num_position
    return data


In [5]:
def recommend_users(company_id, data_company,num_positions,signed_users):
    """
    给定公司id，从满意度数据中选出对该公司满意度最高的num_recommendations个求职者id
    """
    recommendations = []
    for user_id in data_company[company_id]['user_id']:
        if user_id in [u['user_id'] for u in signed_users]:
            continue
        if len(recommendations) < num_positions:
            recommendations.append(user_id)
        else:
            break
    return recommendations

def select_best_offer(user_id, offers):
    """
    从多个offer中选取满意度最高的一个
    """
    max_satisfaction = 2000
    selected_offer = None
    for offer in offers:
        if offer['user_id'] == user_id:
            if offer['satisfaction'] < max_satisfaction:
                max_satisfaction = offer['satisfaction']
                selected_offer = offer
    return selected_offer

def make_offer(user_id, company_id,match, satisfaction):
    """
    构造一个offer
    """
    return {'company_id': company_id,'user_id': user_id,  'match': match, 'satisfaction': satisfaction}
def read_num(num_position,company_id):
    for i in range(len(num_position)):
        if num_position[i][0]==company_id:
            return num_position[i][1]
    return 0
def update_num(num_position,company_id):
    for i in range(len(num_position)):
        if num_position[i][0]==company_id:
            num_position[i][1]-=1
            return num_position
def find_company_position(num_position,company_id):
    for i in range(len(num_position)):
        if num_position[i][0]==company_id:
            return num_position[i][1]
    return 0
def end_ornot(data_company,signed_users,num_position):
    flags=0
    for company_id in data_company:
        recommends=[]
        num1 = find_company_position(num_position,company_id)
        recommends=recommend_users(company_id, data_company, num1,signed_users)
        if len(recommends)>0:
            flags=1
            break
    if flags==0:
        return True
    else:
        return False 
import pandas as pd


In [6]:

# 读取csv文件

# n = job_seekers_df.shape[0] - 1
num_positions = 0

data_seeker = load_seeker_data('data/result3-2.csv')
data_company = load_company_data('data/result3-1+.csv')
# 定义recruit函数


In [7]:
def process_0_position(data_company):
    for company_id in data_company:
        if data_company[company_id]['num_position'] == 0:
            data_company[company_id]['num_position'] = len(data_company[company_id]['user_id'])
    return data_company
data_company = process_0_position(data_company)

In [8]:
num_positions = 0
for company_id in data_company:
    num_positions += int(data_company[company_id]['num_position'])
print(num_positions)

17883


In [9]:
def recruit(data_seeker, data_company,num_positions):
    """
    招聘主函数，给定求职者满意度数据和所需要的人数，返回实际签约的求职者列表和履约率
    """
    signed_users = []
    num_signed_positions = 0
    remaining_positions = num_positions
    total_num_positions = 0
    num_position = [[] for _ in range(len(job_requirement))]
    round=0
    for i,company_id in enumerate(data_company):
        num_position[i].append(company_id)
        num_position[i].append(data_company[company_id]['num_position'])
    while remaining_positions > 0:
        signed_this_round = False
        # 第一轮
        for user_id in data_seeker:
            offer_list = []
            company_id_list = data_seeker[user_id]['company_ids']
            # 如果求职者已经签约，跳过
            if user_id in [u['user_id'] for u in signed_users]:
                continue
            # 获取岗位匹配度非0的公司列表
            # non_zero_companies = [c for c in company_id_list if data.iloc[i, c] > 0]
            for index,c in enumerate(company_id_list):
                num1 = find_company_position(num_position,c)
                recommends=recommend_users(c, data_company, num1,signed_users)
                if user_id in recommends:
                    # 找到与userid对应的满意度
                    satisfaction = data_seeker[user_id]['satisfaction'][index]
                    for user_id1 in data_company[c]['user_id']:
                        if user_id1 == user_id:
                            match = data_company[c]['match'][data_company[c]['user_id'].index(user_id1)]
                            break
                    offer_list.append(make_offer(user_id, c,match,satisfaction))
                    satisfaction+=1
                    total_num_positions += 1
            # 构造offer列表
            # 选取最满意的offer
            best_offer = select_best_offer(user_id, offer_list)

            # 如果最满意的offer已经被其他求职者签约，跳过
            # if best_offer in [u['offer'] for u in signed_users]:
            #     continue

            # 如果该岗位已经招满，跳过
            if best_offer == None:
                continue
            else:
                num = read_num(num_position,best_offer['company_id'])
                if num == 0:
                    continue

            # 签约
            signed_users.append({'user_id': user_id, 'offer': best_offer})
            num_signed_positions += 1
            remaining_positions -= 1
            num_position = update_num(num_position,best_offer['company_id'])
            signed_this_round = True
        print('have delievered ' + str(round) +' round of offers,has signed '+str(num_signed_positions)+' position,still remain '+str(remaining_positions)+' positions')
        round += 1    
        if end_ornot(data_company,signed_users,num_position):
            break
        if not signed_this_round:  # 当前轮次内没有用户签约，终止循环
            break
    # 计算履约率
    completion_rate = num_signed_positions / total_num_positions
    print(num_signed_positions)
    print(total_num_positions)
    return signed_users, completion_rate


In [10]:
print(num_positions)
signed_users, completion_rate = recruit(data_seeker, data_company,num_positions)
print('履约率为：', completion_rate)
print('签约的求职者列表为：')
# 将df的第一行输出为列名 user_id, company_id, satisfaction, match
df = pd.DataFrame(columns=['company_id', 'user_id', 'match', 'satisfaction'])
for u in signed_users:
    # 将u中的offer输出到csv文件中
    print(u['offer'])
    # 将u['offer']从字典转为DataFrame，并写入csv文件
    # 并命名列为user_id, company_id, satisfaction, match
    df = df.append(pd.DataFrame(u['offer'], index=[0]), ignore_index=True)


17883
have delievered 0 round of offers,has signed 120 position,still remain 17763 positions
120
17883
履约率为： 0.006710283509478276
签约的求职者列表为：
{'company_id': 1479021937257742336, 'user_id': 1469222105404014592, 'match': 0.2811076330303003, 'satisfaction': 0.19742137846313082}
{'company_id': 1479021937257742336, 'user_id': 1467792600214929408, 'match': 0.31634964696878637, 'satisfaction': 0.21451548208093735}
{'company_id': 1479024040038498304, 'user_id': 1582598236647063552, 'match': 0.26025500097230486, 'satisfaction': 0.19286520171511864}
{'company_id': 1479021937257742336, 'user_id': 1466004681011625984, 'match': 0.25995262084684684, 'satisfaction': 0.18870272318918918}
{'company_id': 1479024040038498304, 'user_id': 1461534785997504512, 'match': 0.16207859153224577, 'satisfaction': 0.1286454559764197}
{'company_id': 1479021937257742336, 'user_id': 1461648125990141952, 'match': 0.2024525218625775, 'satisfaction': 0.1565709366709787}
{'company_id': 1479021937257742336, 'user_id': 146811

C:\Users\YHWK5\AppData\Local\Temp\ipykernel_13812\3502692647.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(u['offer'], index=[0]), ignore_index=True)
C:\Users\YHWK5\AppData\Local\Temp\ipykernel_13812\3502692647.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(u['offer'], index=[0]), ignore_index=True)
C:\Users\YHWK5\AppData\Local\Temp\ipykernel_13812\3502692647.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(u['offer'], index=[0]), ignore_index=True)
C:\Users\YHWK5\AppData\Local\Temp\ipykernel_13812\3502692647.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [12]:

# rename columns
df.columns = ['招聘信息id', '求职者id', '岗位匹配度', '求职者满意度']
df.to_csv('result4.csv', mode='a', index=False)